In [2]:
import pandas as pd
import numpy as np
from scipy import stats

### Change the currency

UPDATE order_payments
SET payment_value = payment_value / 535;

UPDATE order_items
SET price = price / 535,
	freight_value = freight_value / 535;


## The payment method that spend the most

SELECT SUM(payment_value)/COUNT(*) AS avg_order_value,
	COUNT(*) AS num_transactions,
	STDDEV(payment_value::numeric) AS std,
	payment_type
FROM order_payments
GROUP BY payment_type;

In [19]:
#Th:
mean_bol = 27.10
n_bol = 19784
std_bol = 39.92

mean_cc = 30.52
n_cc = 76795
std_cc = 41.52

mean_dc = 26.64
n_dc = 1529
std_dc = 45.94

mean_vc = 12.27
n_vc = 5775
std_vc = 21.59

alpha = 0.05

#Calculations
st_err_bol = std_bol / np.sqrt(n_bol)
st_err_cc = std_cc / np.sqrt(n_cc)
st_err_dc = std_dc / np.sqrt(n_dc)
st_err_vc = std_vc / np.sqrt(n_vc)
z_crit = stats.norm.ppf(1 - alpha / 2)

print(f'Mean of boleto: {round(mean_bol - st_err_bol*z_crit, 3)} — {round(mean_bol, 3)} — {round(mean_bol + st_err_bol*z_crit, 3)}')
print(f'Mean of credit card: {round(mean_cc - st_err_cc*z_crit, 3)} — {round(mean_cc, 3)} — {round(mean_cc + st_err_cc*z_crit, 3)}')
print(f'Mean of debit card: {round(mean_dc - st_err_dc*z_crit, 3)} — {round(mean_dc, 3)} — {round(mean_dc + st_err_dc*z_crit, 3)}')
print(f'Mean of voucher: {round(mean_vc - st_err_vc*z_crit, 3)} — {round(mean_vc, 3)} — {round(mean_vc + st_err_vc*z_crit, 3)}')


Mean of boleto: 26.544 — 27.1 — 27.656
Mean of credit card: 30.226 — 30.52 — 30.814
Mean of debit card: 24.337 — 26.64 — 28.943
Mean of voucher: 11.713 — 12.27 — 12.827


In [22]:
print(f'Credit c. is {round(100*(30.226 - 27.656)/27.656, 1)}% more than boleto')
print(f'Credit c. is {round(100*(30.226 - 12.827)/12.827, 1)}% more than voucher')

Credit c. is 9.3% more than boleto
Credit c. is 135.6% more than voucher


### Findings:

Customers that use credit card spend 9,3% more than customers that use boleto... and 135.6% more than customers who use voucher.

## Brazilian state more prone to credit_card

WITH freq_table_all_cities AS (
	SELECT COUNT(*) AS frequency,
		customers.customer_state,
		payments.payment_type
	FROM orders
	LEFT JOIN customers ON orders.customer_id = customers.customer_id
	LEFT JOIN order_payments as payments ON payments.order_id = orders.order_id
	WHERE orders.order_status = 'delivered'
		AND payments.payment_type IN ('credit_card', 'boleto')
	GROUP BY customers.customer_state, payments.payment_type
	ORDER BY customers.customer_state
),

freq_table AS(
	SELECT *,
		SUM(frequency) OVER (PARTITION BY customer_state) AS tot_transactions
	FROM freq_table_all_cities
)

SELECT *
FROM freq_table
WHERE tot_transactions > 200;